# MNIST Digit recognition
Steps for building model:
* Preprocess the data
* Build the model architecture
* Augment data
* Train model
* Evaluate model results

The model was able to achieve 99.371% accuracy with 30 epochs, which is in the top 25% for this kaggle competition.

In [32]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import itertools
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/sample_submission.csv


# Loading data
Loading variables train and test with panda dataframe of the training data and testing validation data from csv files.

In [33]:
# format data from csv into dataframe
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")
 

# Formatting
Formatting the data into X_training with pixel values and y_train with labeled. Than delele train because it will no longer be needed.

In [34]:
# Set labels equal to Y_train
y_train = train["label"]
# Drop 'label' column from X_training data
X_train = train.drop(labels = ["label"],axis = 1) 
# free some space
del train

# Normalizing
This stage is just normalizing the 255 greyscale pixel values between 0 and 1.

In [35]:
# Normalize traing and test data
X_train = X_train.astype('float32')
X_train /= 255.
test = test.astype("float32")
test /= 255.

# Reshaping for Convolution
The reshaping adds an additions axis for the greyscale value for the convolution. If this was a color image you would add 3 axis to represent red,green,blue rather than the one we need for the greyscale.

In [37]:
# Reshape input data into 3D for convolution
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

AttributeError: 'DataFrame' object has no attribute 'value'

# One hot-encoding
This one hot-encodes the output values into the one of the 10 output classes. Than splitting the training and testing into 80% for training and 20% for testing.

In [ ]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train, num_classes=10, dtype='float32')

# Split the train and the validation set for the fitting
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, 
                                                    test_size = 0.2, 
                                                    random_state=2)

In [ ]:
epochs = 30
batch_size = 256

# Bulding model
This model is a sequential 10 layer. 
The layers consist of:
* 48,(5x5) 2-D convolutional layer
* 48,(5x5) 2-D convolutional layer
* (2,2) 2-D Max Pooling layer
* 64,(5x5) 2-D convolutional layer
* 64,(5x5) 2-D convolutional layer
* (2,2) 2-D Max Pooling layer
* 32,(5x5) 2-D convolutional layer
* 32,(5x5) 2-D convolutional layer
* Dropout layer (0.25)
* Flatten layer
* 96 Nuron layer with acitivation relu
* Dropout layer (0.25)
* 10 Nuron layer with acitivation softmax

The compiler used for this model consists of the the following:
Loss Function : categorical_crossentropy
Opimizer : adam
Metric : accuracy


In [ ]:
# build model
model = Sequential()

model.add(Conv2D(filters = 48, kernel_size = (5,5),kernel_initializer= 'he_normal', padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 48, kernel_size = (5,5),kernel_initializer= 'he_normal', padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(MaxPooling2D(pool_size = (6,6)))
model.add(Conv2D(filters = 64, kernel_size = (5,5),kernel_initializer= 'he_normal',padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (5,5),kernel_initializer= 'he_normal',padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),kernel_initializer= 'he_normal',padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 32, kernel_size = (5,5),kernel_initializer= 'he_normal',padding = 'Same', 
                 activation ='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(96, activation = "relu", kernel_initializer= 'he_normal'))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax", kernel_initializer = 'glorot_normal'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

# Augmentation
This step augments the dataset to prevent overtfitting. 
The following transformations were used  :
   - Randomly rotate some training images by 10 degrees
   - Randomly  Zoom by 10% some training images
   - Randomly shift images horizontally by 10% of the width
   - Randomly shift images vertically by 10% of the height
   

In [ ]:


datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


In [ ]:
# Fit the model
prediction = model.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_test,y_test),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size
                              )

# Evaluation
## Training and validation curves


In [ ]:
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(prediction.history['loss'], color='b', label="Training loss")
ax[0].plot(prediction.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(prediction.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(prediction.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

# Confusion matrix


In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
Y_pred = model.predict(X_test)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(10)) 

# Display some error results 

In [ ]:

# Errors are difference between predicted labels and true labels
errors = (Y_pred_classes - Y_true != 0)

Y_pred_classes_errors = Y_pred_classes[errors]
Y_pred_errors = Y_pred[errors]
Y_true_errors = Y_true[errors]
X_val_errors = X_test[errors]

def display_errors(errors_index,img_errors,pred_errors, obs_errors):
    """ This function shows 6 images with their predicted and real labels"""
    n = 0
    nrows = 2
    ncols = 3
    fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True)
    for row in range(nrows):
        for col in range(ncols):
            error = errors_index[n]
            ax[row,col].imshow((img_errors[error]).reshape((28,28)))
            ax[row,col].set_title("Predicted label :{}\nTrue label :{}".format(pred_errors[error],obs_errors[error]))
            n += 1

# Probabilities of the wrong predicted numbers
Y_pred_errors_prob = np.max(Y_pred_errors,axis = 1)

# Predicted probabilities of the true values in the error set
true_prob_errors = np.diagonal(np.take(Y_pred_errors, Y_true_errors, axis=1))

# Difference between the probability of the predicted label and the true label
delta_pred_true_errors = Y_pred_errors_prob - true_prob_errors

# Sorted list of the delta prob errors
sorted_dela_errors = np.argsort(delta_pred_true_errors)

# Top 6 errors 
most_important_errors = sorted_dela_errors[-6:]

# Show the top 6 errors
display_errors(most_important_errors, X_val_errors, Y_pred_classes_errors, Y_true_errors)

# predict results

In [ ]:

results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

# Submission

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("mnist_cnn.csv",index=False)